In [3]:
import matplotlib.pyplot as plt
import numpy as np
import scipy
from asp.load import Trial
from asp.cutArray import cutArray
from scipy import interpolate

c:\python27\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [4]:
%matplotlib qt

In [43]:
totalTrial = 12
t = []
for trialId in range(1,totalTrial+1):
    i = trialId-1
    t.append(Trial(1, trialId))
    t[i].readDecoder()
    t[i].readConductor()
    t[i].fitRcurve('knee')
    t[i].fitRcurve('hip')
    t[i].fitRcurve('ankle')

In [52]:
maxdeg = 2*np.pi/totalTrial # degree owned by each trial 
allRcurves = []
allDegs = []
allBars1 = [] # the degree where a bar will be placed to indicate the brainStart
allBars2 = []
for i in range(0, len(t)):
    # temp_r is hip r value in one trial from tdmStart to brainEnd
    temp = cutArray(t[i].rCurveHip, 
                    (t[i].events['tdmStart']-t[i].events['brainStart'])/60, 
                    (t[i].events['brainEnd']-t[i].events['brainStart'])/60)
    temp_r = temp[:,1]
    # temp_deg is the degree on the polar plot
    temp_deg = (temp[:,0]-temp[0,0])/(temp[-1,0]-temp[0,0])*maxdeg + maxdeg*i
    # temp_bar is the degree where brainStart bar will be plotted
    temp_bar1 = (0-temp[0,0])/(temp[-1,0]-temp[0,0])*maxdeg + maxdeg*i
    # temp_bar2 is the bars that seperate trials
    temp_bar2 = maxdeg*i
    
    allRcurves.append(temp_r)
    allDegs.append(temp_deg)
    allBars1.append(temp_bar1)
    allBars2.append(temp_bar2)

In [94]:
fig = plt.figure(figsize=(7,5.5))

ax = fig.add_subplot(111, projection='polar')

for i in range(0, len(t)):
    ax.plot(allDegs[i], allRcurves[i],color='#1f77b4')  
    ax.plot([allBars1[i],allBars1[i]], [0,0.5],'k--',linewidth=0.3)
    ax.plot([allBars2[i],allBars2[i]], [0,0.6],'k',linewidth=0.8)

ax.set_rticks([0, 0.3, 0.6])  # less radial ticks
ax.set_rmax(0.6)
ax.grid(False)
ax.set_rlabel_position(90)  # get radial labels away from plotted line
ax.set_xticks(np.array(allBars2)+ 2*np.pi/totalTrial/2)
xtickLabels = []
for i in range(0, len(t)):
    xtickLabels.append('Session '+str(i+1))


font = {'family' : 'serif',
        'weight' : 'normal'
#         'size'   : 22
       }
plt.rc('font', **font)

ax.set_xticklabels(xtickLabels, fontname='serif',fontweight='bold')

In [95]:
plt.savefig('Figures/polar_r_values_SS01.png',dpi=300,pad_inches=0, transparent=True)